# Graphic Equalizer in Praat-Parselmouth

This code is translated from:
http://www.praatvocaltoolkit.com/eq-10-bands.html


## With sliders...

In [1]:
import ipywidgets
import glob
import parselmouth
from parselmouth.praat import call
from parselmouth.praat import run

In [2]:
def eq_band(band1, band2, db, smoothing, buffer, sp_eq, sp_pulse):
    amp = 0.00002 * 10 ** (db / 20)
    run([buffer, sp_pulse], """
selectObject: 1
Formula: "object[""Spectrum sp_pulse""]"
""")

    call(buffer, "Filter (pass Hann band)", band1, band2, smoothing)
 
    call(buffer, "Formula", f"self * {amp}")

    run([sp_eq, buffer], """
selectObject: 1
Formula: "self + object[""Spectrum buffer""]"
""")
    

def graphic_equalizer(audio_file,
                      band_1__31_5_Hz,
                      band_2__63_Hz,
                      band_3__125_Hz,
                      band_4__250_Hz,
                      band_5__500_Hz,
                      band_6__1000_Hz,
                      band_7__2000_Hz,
                      band_8__4000_Hz,
                      band_9__8000_Hz,
                      band_10__16000_Hz,
                     ):
    sound = parselmouth.Sound(audio_file)
    duration = sound.get_total_duration()
    intensity = call(sound, "Get intensity (dB)")
    sf = sound.sampling_frequency
    pointprocess = call("Create empty PointProcess", "pulse", 0, 0.05)
    call(pointprocess, "Add point", 0.025)
    pulse = call(pointprocess, "To Sound (pulse train)", sf, 1, 0.05, 2000)
    sp_pulse = call(pulse, "To Spectrum", "no")
    sp_pulse.name = "sp_pulse"
    buffer = sp_pulse.copy()
    buffer.name = "buffer"
    call(buffer, "Formula", "0")
    sp_eq = buffer.copy()
    sp_eq.name = "sp_eq"
    eq_band(0, 44.2, band_1__31_5_Hz, 20, buffer, sp_eq, sp_pulse)
    eq_band(44.2, 88.4, band_2__63_Hz, 20, buffer, sp_eq, sp_pulse)
    eq_band(88.4, 177, band_3__125_Hz, 40, buffer, sp_eq, sp_pulse)
    eq_band(177, 354, band_4__250_Hz, 80, buffer, sp_eq, sp_pulse)
    eq_band(354, 707, band_5__500_Hz, 100, buffer, sp_eq, sp_pulse)
    eq_band(707, 1414, band_6__1000_Hz, 100, buffer, sp_eq, sp_pulse)
    eq_band(1414, 2828, band_7__2000_Hz, 100, buffer, sp_eq, sp_pulse)
    eq_band(2828, 5657, band_8__4000_Hz, 100, buffer, sp_eq, sp_pulse)
    eq_band(5657, 11314, band_9__8000_Hz, 100, buffer, sp_eq, sp_pulse)
    eq_band(11314, max(12000, sf / 2), band_10__16000_Hz, 100, buffer, sp_eq, sp_pulse)
    call(sp_eq, "Filter (pass Hann band)", 80, 0, 20)
    call(sp_eq, "Filter (pass Hann band)", 0, 20000, 100)
    pulse_eq_temp = sp_eq.to_sound()
    dur2 = pulse_eq_temp.get_total_duration()
    pulse_eq = call(pulse_eq_temp, "Extract part", (dur2 - 0.05) / 2, dur2 - ((dur2 - 0.05) / 2), "Hanning", 1, "no")
    pulse_eq.scale_peak(99)
    pulse_sf = pulse_eq.sampling_frequency
    if pulse_sf != sf:
        call(pulse_eq, "Override sampling frequency", sf)
    tmp1 = call([pulse_eq, sound], "Convolve", "sum", "zero")
    tmp2 = call(tmp1, "Extract part", 0.025, duration + 0.025, "rectangular", 1, "no")
    return Audio(data=tmp2.values, rate=tmp2.sampling_frequency)
    

In [5]:
from IPython.display import Audio

w = ipywidgets.interact(graphic_equalizer,
                        audio_file=ipywidgets.Dropdown(options=sorted(glob.glob("*.wav")), value="03-01-01-01-01-01-01.wav"),
                        band_1__31_5_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_2__63_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_3__125_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_4__250_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_5__500_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_6__1000_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_7__2000_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_8__4000_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_9__8000_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0),
                        band_10__16000_Hz=ipywidgets.IntSlider(min=-24, max=24, step=1, value=0))
            


TraitError: Invalid selection: value not found

# Here's the code breakdown

# Import modules

In [ ]:
import parselmouth
from parselmouth.praat import call
from parselmouth.praat import run

# Load Soud

In [ ]:
sound = parselmouth.Sound("03-01-01-01-01-01-01.wav")

# Set values for graphic equalizer

In [ ]:
band_1__31_5_Hz = 24  # this number is dB
band_2__63_Hz = 18
band_3__125_Hz = 12
band_4__250_Hz = 6
band_5__500_Hz = 4
band_6__1000_Hz = 0
band_7__2000_Hz = -6
band_8__4000_Hz = -12
band_9__8000_Hz = -18
band_10__16000_Hz = -24

## Round these values to make sure they're integers in the right range

In [ ]:
band_1__31_5_Hz = min(max(round(band_1__31_5_Hz), -24), 24)
band_2__63_Hz = min(max(round(band_2__63_Hz), -24), 24)
band_3__125_Hz = min(max(round(band_3__125_Hz), -24), 24)
band_4__250_Hz = min(max(round(band_4__250_Hz), -24), 24)
band_5__500_Hz = min(max(round(band_5__500_Hz), -24), 24)
band_6__1000_Hz = min(max(round(band_6__1000_Hz), -24), 24)
band_7__2000_Hz = min(max(round(band_7__2000_Hz), -24), 24)
band_8__4000_Hz = min(max(round(band_8__4000_Hz), -24), 24)
band_9__8000_Hz = min(max(round(band_9__8000_Hz), -24), 24)
band_10__16000_Hz = min(max(round(band_10__16000_Hz), -24), 24)

# Measure Sound
- Duration
- Intensity
- Sampling Rate

In [ ]:
duration = sound.get_total_duration()
intensity = call(sound, "Get intensity (dB)")
sf = sound.sampling_frequency

# Create spectral objects based on a pulse train of the sound

In [ ]:
pointprocess = call("Create empty PointProcess", "pulse", 0, 0.05)
call(pointprocess, "Add point", 0.025)
pulse = call(pointprocess, "To Sound (pulse train)", sf, 1, 0.05, 2000)
sp_pulse = call(pulse, "To Spectrum", "no")
sp_pulse.name = "sp_pulse"
buffer = sp_pulse.copy()
buffer.name = "buffer"
call(buffer, "Formula", "0")
sp_eq = buffer.copy()
sp_eq.name = "sp_eq"

# Function to modify amplitudes in a particular band

In [ ]:
def eq_band(band1, band2, db, smoothing):
    amp = 0.00002 * 10 ** (db / 20)
    
    run([buffer, sp_pulse], """
selectObject: 1
Formula: "object[""Spectrum sp_pulse""]"
""")

    call(buffer, "Filter (pass Hann band)", band1, band2, smoothing)
 
    call(buffer, "Formula", f"self * {amp}")

    run([sp_eq, buffer], """
selectObject: 1
Formula: "self + object[""Spectrum buffer""]"
""")
    

# Modify band amplitudes

In [ ]:
eq_band(0, 44.2, band_1__31_5_Hz, 20)
eq_band(44.2, 88.4, band_2__63_Hz, 20)
eq_band(88.4, 177, band_3__125_Hz, 40)
eq_band(177, 354, band_4__250_Hz, 80)
eq_band(354, 707, band_5__500_Hz, 100)
eq_band(707, 1414, band_6__1000_Hz, 100)
eq_band(1414, 2828, band_7__2000_Hz, 100)
eq_band(2828, 5657, band_8__4000_Hz, 100)
eq_band(5657, 11314, band_9__8000_Hz, 100)
eq_band(11314, max(12000, sf / 2), band_10__16000_Hz, 100)


call(sp_eq, "Filter (pass Hann band)", 80, 0, 20)
call(sp_eq, "Filter (pass Hann band)", 0, 20000, 100)


# Convert band filtered spectrum into a sound and measure it's duration

In [ ]:
pulse_eq_temp = sp_eq.to_sound()
dur2 = pulse_eq_temp.get_total_duration()

# Trim the amplitude filtered sound & scale the amplitude

In [ ]:
pulse_eq = call(pulse_eq_temp, "Extract part", (dur2 - 0.05) / 2, dur2 - ((dur2 - 0.05) / 2), "Hanning", 1, "no")
pulse_eq.scale_peak(99)
pulse_sf = pulse_eq.sampling_frequency
if pulse_sf != sf:
    call(pulse_eq, "Override sampling frequency", sf)

# Convolve the amplitude filtered sound and the original sound

In [ ]:
tmp1 = call([pulse_eq, sound], "Convolve", "sum", "zero")

# Trim the final sound

In [ ]:
tmp2 = call(tmp1, "Extract part", 0.025, duration + 0.025, "rectangular", 1, "no")

# Play the sound

In [ ]:
from IPython.display import Audio
Audio(data=tmp2.values, rate=tmp2.sampling_frequency)